<a href="https://colab.research.google.com/github/mohit712319/EECS448Lab10/blob/main/NNLSsolver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# Upload the file
uploaded = files.upload()

In [ ]:
#Data matrix

from scipy.io import loadmat

data = loadmat('u_out_x1.mat')
#data2=loadmat('dof_y_x.mat')
# Print all keys in the .mat file
print("Keys in the .mat file:", data.keys())
#print("Keys in the .mat file:", data2.keys())

# Access a specific variable (replace 'variable_name' with the actual key)
var = data['u_out_x']
#var2=data2['dof_y_x']

# Check the shape of the data
print("Shape of the variable:", var.shape)
#print("Shape of the variable2:", var2.shape)


# Print a chunk of the data (e.g., first 10 rows and columns)
chunk = var[:3000, :10]  # Adjust the slicing as needed
print("Chunk of the data:")
print(chunk)


def is_positive_definite(matrix):
    try:
        np.linalg.cholesky(matrix)
        return True
    except np.linalg.LinAlgError:
        return False
is_positive_definite(var)

Keys in the .mat file: dict_keys(['__header__', '__version__', '__globals__', 'u_out_x'])
Shape of the variable: (7613, 300)
Chunk of the data:
[[1.49417435 1.49415662 1.49415658 ... 1.49415732 1.49415752 1.49415773]
 [1.49998177 1.49996368 1.49996501 ... 1.49997442 1.49997632 1.49997748]
 [1.49094999 1.49088654 1.49084297 ... 1.49075427 1.49076597 1.49078476]
 ...
 [0.9174007  0.86938812 0.80742092 ... 0.4755     0.44413207 0.43177124]
 [0.72629028 0.69902746 0.66170518 ... 0.44119187 0.41831745 0.40759783]
 [0.63241571 0.61455627 0.58925602 ... 0.40832831 0.3816993  0.36402686]]


False

Current Problem:Selecting subset of columns from full matrix A and C so BC>0 min:||A-BC|| so BC>=0

New Problem:Minimize ||S(A-BC)|| so BC>=0

Write some helper functions to clean up code and resolve Forbenius norm error
Look for faster NNLS solver for mult right side cols>>rows


Update:Sketching seems to throw of NNLS solvers even if matrix is same size as random amtrix.



In [ ]:
from re import X
import numpy as np
import scipy.linalg as sla
from numpy.linalg import norm
from sklearn.metrics import r2_score
from scipy.linalg import qr,solve_triangular
from scipy.optimize import nnls
import cvxpy as cp
import time

rng = np.random.default_rng(123)



def reconst(Q,R,d):
    lambda_star, _ = nnls(Q.T, d)
    y = d - (0.5*(Q.T @ lambda_star))
    #y=Q.T@lambda_star
    x = solve_triangular(R, y, lower=False)
    return(x)

m=7000
n=300
m_small=2000
A   = rng.standard_normal(size=(m,n)) # Example data matrix
S = rng.standard_normal(size=(m_small,m))
A_sketch=S@A
m, n = A_sketch.shape

b=np.random.rand(m,1)
sol,rnorm=nnls(A_sketch,b.flatten())
print(np.linalg.norm((A_sketch@sol)-b))

# Randomly select k column indices from A
k=60
col_indices = np.random.choice(n, k, replace=False)
B = A[:, col_indices]  # Subset of columns from A
B_sketch=S@B
start_time = time.time()

Q, R = qr(B_sketch, mode='economic')  # A = QR


C_star = np.zeros((k, n))


for j in range(n):
    d =(2*Q.T @ A_sketch[:,j]) # Compute d columns of A behave as b
    x_star=reconst(Q,R,d)
    C_star[:, j]= x_star



A_sketchapprox=B_sketch@C_star
error = np.linalg.norm(A_sketch - A_sketchapprox, 'fro')
print(f"Reconstruction error (Frobenius norm): {error:.4f}")
print(f"Minimum value of B@C: {np.min(np.ravel(B_sketch@C_star))}")
end_time = time.time()
#print(end_time-start_time)
print(f"Condition number of sketched A: {np.linalg.cond(A_sketch)}")









#Testing with random matrix of same size as A_sketch
# QR decomposition of A
A_sketch=np.random.rand(m, n)
b=np.random.rand(m,1)
sol,rnorm=nnls(A_sketch,b.flatten())
print(np.linalg.norm((A_sketch@sol)-b))

m, n = A_sketch.shape
# Randomly select k column indices from A
k=60
col_indices = np.random.choice(n, k, replace=False)
B_sketch = A_sketch[:, col_indices]  # Subset of columns from A
start_time = time.time()

Q, R = qr(B_sketch, mode='economic')  # A = QR


C_star = np.zeros((k, n))



for j in range(n):
    d =(2*Q.T @ A_sketch[:,j]) # Compute d columns of A behave as b
    x_star=reconst(Q,R,d)
    C_star[:, j]= x_star


#A_approx = B @ C_star #B is not defined


A_sketchapprox=B_sketch@C_star
error = np.linalg.norm(A_sketch - A_sketchapprox, 'fro')
print("Info for random matrix ")
print(f"Reconstruction error (Frobenius norm): {error:.4f}")
print(f"Minimum value of B@C: {np.min(np.ravel(B_sketch@C_star))}")
end_time = time.time()
#print(end_time-start_time)
print(f"Condition number of sketched A: {np.linalg.cond(A_sketch)}")




1137.6356875505398
Reconstruction error (Frobenius norm): 56787.4298
Minimum value of B@C: -360.678066868067
Condition number of sketched A: 2.5301304584922435
572.5224840143245
Info for random matrix 
Reconstruction error (Frobenius norm): 198.6001
Minimum value of B@C: 6.290947759125985e-07
Condition number of sketched A: 48.10368977067608


In [ ]:
#Try With faster NNLS

A_sketch=np.random.rand(1000, 300)
m=1000
n=300
# Randomly select k column indices from A
k=60
col_indices = np.random.choice(n, k, replace=False)
B_sketch = A_sketch[:, col_indices]  # Subset of columns from A
start_time = time.time()

Q, R = qr(B_sketch, mode='economic')  # A = QR


C_star = np.zeros((k, n))



for j in range(n):
    d =(2*Q.T @ A_sketch[:,j]) # Compute d columns of A behave as b
    x_star=reconst(Q,R,d)
    C_star[:, j]= x_star


A_approx = B @ C_star


**Testing Sketches**

## Stephen's experiment, Mon Feb 10 2025

In [ ]:
import numpy as np
import scipy.linalg as sla
from numpy.linalg import norm
from sklearn.metrics import r2_score

In [ ]:
rng = np.random.default_rng(123)

m   = int(7e3)
n   = 300
A   = rng.standard_normal(size=(m,n))
# We have a random design matrix, but let's not just pick a random RHS, let's pick a RHS associated with some "xTrue"
xTrue   = rng.standard_normal(size=(n,1))
b   = A@xTrue
# add a bit of noise
b   = b + .01*rng.standard_normal(size=(m,1))*norm(b)/np.sqrt(m)

# Alternatively, just pick b randomly, so there might not be any good solution, i.e., rel res is .977, and R^2 is near zero
# b   = rng.standard_normal(size=(m,1))

relErr  = lambda x : norm(A@x-b) / norm(b) # relative squared error of residual
R2      = lambda x : 1 - relErr(x)**2      # "R^2" aka https://en.wikipedia.org/wiki/Coefficient_of_determination

xLS,res = sla.lstsq(A,b)[:2] # res = norm(A@xLS-b)**2
print(f'Basic least squares, relative residual is {relErr(xLS):.2e} (lower is better)')
errLS = relErr(xLS)

# and some other metrics
# R2(xLS), r2_score(b,A@xLS) # equivalent ways to do it
print(f'  and R^2 is {R2(xLS):.4f} (higher, i.e., closer to 1, is better)')

Basic least squares, relative residual is 9.88e-03 (lower is better)
  and R^2 is 0.9999 (higher, i.e., closer to 1, is better)


In [ ]:
small_m_list = np.logspace(2,4,10, dtype=int)
for m_small in small_m_list:
  S         = rng.standard_normal(size=(m_small,m)) # normalization doesn't matter in this context
  xSketched = sla.lstsq(S@A,S@b)[0]
  err       = relErr(xSketched)
  print(f'Sketched S with m={m_small:5d} i.e. {m_small/m*100:5.1f}%, rel. resid. is {err:.2e} i.e. {err/errLS:.2f} fraction of unsketched')

Sketched S with m=  100 i.e.   1.4%, rel. resid. is 7.80e-01 i.e. 78.91 fraction of unsketched
Sketched S with m=  166 i.e.   2.4%, rel. resid. is 6.58e-01 i.e. 66.65 fraction of unsketched
Sketched S with m=  278 i.e.   4.0%, rel. resid. is 2.67e-01 i.e. 27.03 fraction of unsketched
Sketched S with m=  464 i.e.   6.6%, rel. resid. is 1.67e-02 i.e. 1.69 fraction of unsketched
Sketched S with m=  774 i.e.  11.1%, rel. resid. is 1.22e-02 i.e. 1.23 fraction of unsketched
Sketched S with m= 1291 i.e.  18.4%, rel. resid. is 1.11e-02 i.e. 1.13 fraction of unsketched
Sketched S with m= 2154 i.e.  30.8%, rel. resid. is 1.06e-02 i.e. 1.08 fraction of unsketched
Sketched S with m= 3593 i.e.  51.3%, rel. resid. is 1.03e-02 i.e. 1.04 fraction of unsketched
Sketched S with m= 5994 i.e.  85.6%, rel. resid. is 1.02e-02 i.e. 1.03 fraction of unsketched
Sketched S with m=10000 i.e. 142.9%, rel. resid. is 1.00e-02 i.e. 1.02 fraction of unsketched


In [ ]:
from re import X
import numpy as np
import scipy.linalg as sla
from numpy.linalg import norm
from sklearn.metrics import r2_score
from scipy.linalg import qr,solve_triangular
from scipy.optimize import nnls
import cvxpy as cp
import time

rng = np.random.default_rng(123)



def reconst(Q,R,d):
    lambda_star, _ = nnls(Q.T, d)
    y = d - (0.5*(Q.T @ lambda_star))
    #y=Q.T@lambda_star
    x = solve_triangular(R, y, lower=False)
    return(x)



m   = int(7e3)
n   = 300
k=60

# Set random seed
rng = np.random.default_rng(seed=42)

A   = rng.standard_normal(size=(m,n))
# We have a random design matrix, but let's not just pick a random RHS, let's pick a RHS associated with some "xTrue"
col_indices = rng.choice(n, k, replace=False)
B = A[:, col_indices]
X_true = rng.standard_normal(size=(k, n))
A  = B@X_true
# Add noise to A(behaving as b)
A=A+ 0.01 * rng.standard_normal(size=(m, n)) * np.linalg.norm(A) / np.sqrt(m)

X_LS=np.zeros((k,n))
relErr = lambda x: np.linalg.norm(A - B @ x, 'fro') / np.linalg.norm(A, 'fro')
R2      = lambda x : 1 - relErr(x)**2



#Solve least squares problem
for j in range(n):
  x_col, res, _, _ = sla.lstsq(B, A[:,j])
  X_LS[:,j] = x_col.flatten()

#print errors
print(np.linalg.norm(A - B @ X_LS, 'fro'))
print(f'Basic least squares, relative residual is {relErr(X_LS):.2e} (lower is better)')
errLS = relErr(X_LS)

# and some other metrics
print(f'  and R^2 is {R2(X_LS):.4f} (higher, i.e., closer to 1, is better)')






m_small=2000
S = rng.standard_normal(size=(m_small,m)) # normalization doesn't matter in this context
xSketched=np.zeros((k,n))
for j in range(n):
  x_col, res, _, _ = sla.lstsq(S@B, S@A[:,j])
  xSketched[:,j] = x_col.flatten()
err    = relErr(xSketched)
print(f'Sketched S with m={m_small:5d} i.e. {m_small/m*100:5.1f}%, rel. resid. is {err:.2e} i.e. {err/errLS:.2f} fraction of unsketched')

1952.0946999168395
Basic least squares, relative residual is 1.70e-01 (lower is better)
  and R^2 is 0.9712 (higher, i.e., closer to 1, is better)
Sketched S with m= 2000 i.e.  28.6%, rel. resid. is 1.72e-01 i.e. 1.02 fraction of unsketched


In [ ]:
small_m_list = np.logspace(2,3,10, dtype=int)
for m_small in small_m_list:
  S         = rng.standard_normal(size=(m_small,m)) # normalization doesn't matter in this context
  xSketched=np.zeros((k,n))
  for j in range(n):
    x_col, res, _, _ = sla.lstsq(S@B, S@A[:,j])
    xSketched[:,j] = x_col.flatten()
  err    = relErr(xSketched)
  print(f'Sketched S with m={m_small:5d} i.e. {m_small/m*100:5.1f}%, rel. resid. is {err:.2e} i.e. {err/errLS:.2f} fraction of unsketched')

Sketched S with m=  100 i.e.   1.4%, rel. resid. is 2.83e-01 i.e. 1.66 fraction of unsketched
Sketched S with m=  129 i.e.   1.8%, rel. resid. is 2.31e-01 i.e. 1.36 fraction of unsketched
Sketched S with m=  166 i.e.   2.4%, rel. resid. is 2.13e-01 i.e. 1.25 fraction of unsketched
Sketched S with m=  215 i.e.   3.1%, rel. resid. is 2.00e-01 i.e. 1.18 fraction of unsketched
Sketched S with m=  278 i.e.   4.0%, rel. resid. is 1.92e-01 i.e. 1.13 fraction of unsketched
Sketched S with m=  359 i.e.   5.1%, rel. resid. is 1.86e-01 i.e. 1.10 fraction of unsketched
Sketched S with m=  464 i.e.   6.6%, rel. resid. is 1.82e-01 i.e. 1.07 fraction of unsketched
Sketched S with m=  599 i.e.   8.6%, rel. resid. is 1.79e-01 i.e. 1.05 fraction of unsketched
Sketched S with m=  774 i.e.  11.1%, rel. resid. is 1.77e-01 i.e. 1.04 fraction of unsketched
Sketched S with m= 1000 i.e.  14.3%, rel. resid. is 1.75e-01 i.e. 1.03 fraction of unsketched


#Same approach with NNLS

In [ ]:
#Same approach with NNLS
from re import X
import numpy as np
import scipy.linalg as sla
from numpy.linalg import norm
from sklearn.metrics import r2_score
from scipy.linalg import qr,solve_triangular
from scipy.optimize import nnls
import cvxpy as cp
import time

rng = np.random.default_rng(123)



def reconst(Q,R,d):
    lambda_star, _ = nnls(Q.T, d)
    y = d - (0.5*(Q.T @ lambda_star))
    #y=Q.T@lambda_star
    x = solve_triangular(R, y, lower=False)
    return(x)



m   = int(7e3)
n   = 300
k=60

# Set random seed
rng = np.random.default_rng(seed=42)

A   = rng.standard_normal(size=(m,n))
# We have a random design matrix, but let's not just pick a random RHS, let's pick a RHS associated with some "xTrue"
col_indices = rng.choice(n, k, replace=False)
B = A[:, col_indices]
X_true = rng.standard_normal(size=(k, n))
A  = B@X_true
# Add noise to A(behaving as b)
A=A+ 0.01 * rng.standard_normal(size=(m, n)) * np.linalg.norm(A) / np.sqrt(m)

X_LS=np.zeros((k,n))
relErr = lambda x: np.linalg.norm(A - B @ x, 'fro') / np.linalg.norm(A, 'fro')
R2      = lambda x : 1 - relErr(x)**2



#Solve least squares problem(NNLS)
Q, R = qr(B, mode='economic')  # A = QR



for j in range(n):
    x_star,_=nnls(B,A[:,j])
    X_LS[:, j]= x_star


#print errors
print(np.linalg.norm(A - B @ X_LS, 'fro'))
print(f'Basic least squares, relative residual is {relErr(X_LS):.2e} (lower is better)')
errLS = relErr(X_LS)

# and some other metrics
print(f'  and R^2 is {R2(X_LS):.4f} (higher, i.e., closer to 1, is better)')






m_small=2000
S = rng.standard_normal(size=(m_small,m)) # normalization doesn't matter in this context
xSketched=np.zeros((k,n))
for j in range(n):
  x_col, res, _, _ = sla.lstsq(S@B, S@A[:,j])
  xSketched[:,j] = x_col.flatten()
err    = relErr(xSketched)
print(f'Sketched S with m={m_small:5d} i.e. {m_small/m*100:5.1f}%, rel. resid. is {err:.2e} i.e. {err/errLS:.2f} fraction of unsketched')


8234.83332650032
Basic least squares, relative residual is 7.17e-01 (lower is better)
  and R^2 is 0.4866 (higher, i.e., closer to 1, is better)
Sketched S with m= 2000 i.e.  28.6%, rel. resid. is 1.72e-01 i.e. 0.24 fraction of unsketched


In [ ]:
small_m_list = np.logspace(2,3,10, dtype=int)
for m_small in small_m_list:
  S         = rng.standard_normal(size=(m_small,m)) # normalization doesn't matter in this context
  xSketched=np.zeros((k,n))
  for j in range(n):
    x_col, _ = nnls(S@B, S@A[:,j])
    xSketched[:,j] = x_col.flatten()
  err    = relErr(xSketched)
  print(f'Sketched S with m={m_small:5d} i.e. {m_small/m*100:5.1f}%, rel. resid. is {err:.2e} i.e. {err/errLS:.2f} fraction of unsketched')

Sketched S with m=  100 i.e.   1.4%, rel. resid. is 8.40e-01 i.e. 1.17 fraction of unsketched
Sketched S with m=  129 i.e.   1.8%, rel. resid. is 8.09e-01 i.e. 1.13 fraction of unsketched
Sketched S with m=  166 i.e.   2.4%, rel. resid. is 7.81e-01 i.e. 1.09 fraction of unsketched
Sketched S with m=  215 i.e.   3.1%, rel. resid. is 7.73e-01 i.e. 1.08 fraction of unsketched
Sketched S with m=  278 i.e.   4.0%, rel. resid. is 7.59e-01 i.e. 1.06 fraction of unsketched
Sketched S with m=  359 i.e.   5.1%, rel. resid. is 7.46e-01 i.e. 1.04 fraction of unsketched
Sketched S with m=  464 i.e.   6.6%, rel. resid. is 7.37e-01 i.e. 1.03 fraction of unsketched
Sketched S with m=  599 i.e.   8.6%, rel. resid. is 7.34e-01 i.e. 1.02 fraction of unsketched
Sketched S with m=  774 i.e.  11.1%, rel. resid. is 7.32e-01 i.e. 1.02 fraction of unsketched
Sketched S with m= 1000 i.e.  14.3%, rel. resid. is 7.27e-01 i.e. 1.02 fraction of unsketched
